# **Tokenization**
La tokenización es el proceso mediante el cual los inputs de texto de un modelo de lenguaje se transforman a una representación numérica.

## **Strings en Python**
Los [strings en Python](https://docs.python.org/3/library/stdtypes.html#text-sequence-type-str) son secuencias innumables de valores codificados en [unicode](https://es.wikipedia.org/wiki/Unicode). Se puede acceder utilizando `ord()`

In [1]:
ord("h")

104

In [2]:
[ord(x) for x in "Hello World"]

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100]

Unicode tiene tres manera de codificar estos valores en unicode y traducirlos a bytes: UTF-8,UTF-16, UTF-32... Lo que hacen es traducir todos los códigos de Unicode a valores de 4 bytes. UTF-8 es el que más se utiliza porque tiene retrocompatibilidad con ASCII.

In [3]:
list("Hello World".encode("utf-8"))

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100]

El problema de utilizar esto es que nuestras secuencias de texto serán largísimas. Para resolver esto acudimos al algoritmo [Byte Pair Encoding](https://en.wikipedia.org/wiki/Byte-pair_encoding) para comprimir las secuencias de bytes. Byte Pair Encoding lo que hace es ir iterando sobre la cadena de texto encontrando los pares que caracteres que más se repiten y sustituyendolos por un token nuevo.

```
aaabdaaabac
```

Se sustituyen las `aa` por `Z`:
```
ZabdZabac
Z=aa
```

Se sustituyen las `ab` por `Y`:
```
ZYdZYac
Y=ab
Z=aa
```

Se sustituyen las `ZY` por `X`:
```
XdXac
X=ZY
Y=ab
Z=aa
```

Aplicación:

In [4]:
text = """En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor."""

tokens = text.encode("utf8")
tokens = list(map(int, tokens))

print("---")
print(text)
print(f"Número de caracteres: {len(text)}")

print("\n---")
print(tokens)
print(f"Número de bytes {len(tokens)}")

---
En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor.
Número de caracteres: 177

---
[69, 110, 32, 117, 110, 32, 108, 117, 103, 97, 114, 32, 100, 101, 32, 108, 97, 32, 77, 97, 110, 99, 104, 97, 44, 32, 100, 101, 32, 99, 117, 121, 111, 32, 110, 111, 109, 98, 114, 101, 32, 110, 111, 32, 113, 117, 105, 101, 114, 111, 32, 97, 99, 111, 114, 100, 97, 114, 109, 101, 44, 32, 110, 111, 32, 104, 97, 32, 109, 117, 99, 104, 111, 32, 116, 105, 101, 109, 112, 111, 32, 113, 117, 101, 32, 118, 105, 118, 195, 173, 97, 32, 117, 110, 32, 104, 105, 100, 97, 108, 103, 111, 32, 100, 101, 32, 108, 111, 115, 32, 100, 101, 32, 108, 97, 110, 122, 97, 32, 101, 110, 32, 97, 115, 116, 105, 108, 108, 101, 114, 111, 44, 32, 97, 100, 97, 114, 103, 97, 32, 97, 110, 116, 105, 103, 117, 97, 44, 32, 114, 111, 99, 195, 173, 110, 32, 102, 108, 97, 99, 111, 32, 121, 32, 103, 97, 108, 103, 111, 32, 

Hay más bytes que caracteres ya que los bytes a veces se unen para combertirse en caracteres (cómo las tíldes, por ejemplo). Ahora queremos iterar y encontrar el número de bytes que se repite más a menudo.

In [5]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
print(sorted(((v, k) for k, v in stats.items()), reverse = True))

[(9, (111, 32)), (6, (101, 32)), (5, (110, 32)), (5, (97, 32)), (4, (100, 101)), (4, (44, 32)), (4, (32, 108)), (4, (32, 100)), (4, (32, 97)), (3, (116, 105)), (3, (114, 111)), (3, (111, 114)), (3, (110, 111)), (3, (108, 97)), (3, (103, 97)), (3, (100, 97)), (3, (99, 111)), (3, (97, 114)), (3, (97, 110)), (3, (32, 110)), (2, (195, 173)), (2, (117, 110)), (2, (114, 101)), (2, (113, 117)), (2, (108, 103)), (2, (105, 101)), (2, (104, 97)), (2, (103, 111)), (2, (101, 114)), (2, (99, 104)), (2, (97, 108)), (2, (97, 99)), (2, (97, 44)), (2, (32, 117)), (2, (32, 113)), (2, (32, 104)), (2, (32, 99)), (1, (173, 110)), (1, (173, 97)), (1, (122, 97)), (1, (121, 111)), (1, (121, 32)), (1, (118, 195)), (1, (118, 105)), (1, (117, 121)), (1, (117, 105)), (1, (117, 103)), (1, (117, 101)), (1, (117, 99)), (1, (117, 97)), (1, (115, 116)), (1, (115, 32)), (1, (114, 114)), (1, (114, 109)), (1, (114, 103)), (1, (114, 100)), (1, (114, 46)), (1, (114, 32)), (1, (112, 111)), (1, (111, 115)), (1, (111, 109)), 

Aquí tenemos una lista con las veces que se repiten cada uno de los pares, en nuestro caso el que más se repite es el par compuesto de `111`, `32`.

In [6]:
top_pair = max(stats, key = stats.get)
top_pair

(111, 32)

In [7]:
chr(111), chr(32)

('o', ' ')

Parece que tenemos muchos `o ` es decir, la letra o seguido de un espacio. Ahora lo que debemos hacer es asignarle un token, que será el 256, ya que hasta el 255 ya está ocupado. Esto porque la codificación UTF-8 va del 0 al 255 (8 bits).

In [8]:
def merge(ids, pair, idx):
    newids = list()
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

tokens2 = merge(tokens, top_pair, 256)
print(len(tokens2))

170


Ha salido 170 de longitud, que coincide con 179 que había antes - 9 repetidos del par mayor. Ahora ya podemos hacer un bucle que haga el trabajo por nosotros. Sin embargo, surge la pregunta de... *¿Hasta cuando lo hacemos?* A medida que vayamos iterando, nuestro vocabulario de tokens irá aumentando y nuestro length ira disminuyendo. El objetivo es encontrar un "sweetspot". Podemos elegir un tamaño de vocabulario y el número de iteraciones será...

$$NIterations = DesiredSize - 256 $$

Comenzamos con 256 (del 0 al 255 por los 8 bits de UTF8) y queremos añadir hasta llegar a nuestro tamaño de vocabulario.

In [9]:
vocab_size = 276
num_merges = vocab_size - 256

ids = list(tokens)

merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key = stats.get)
    idx = 256 + i
    print(f"Merging {pair} into a new token {idx}")

    ids = merge(ids, pair, idx)
    merges[pair] = idx

Merging (111, 32) into a new token 256
Merging (101, 32) into a new token 257
Merging (110, 32) into a new token 258
Merging (97, 32) into a new token 259
Merging (100, 257) into a new token 260
Merging (44, 32) into a new token 261
Merging (97, 114) into a new token 262
Merging (260, 108) into a new token 263
Merging (97, 110) into a new token 264
Merging (111, 114) into a new token 265
Merging (116, 105) into a new token 266
Merging (117, 258) into a new token 267
Merging (32, 263) into a new token 268
Merging (99, 104) into a new token 269
Merging (97, 261) into a new token 270
Merging (110, 256) into a new token 271
Merging (113, 117) into a new token 272
Merging (101, 114) into a new token 273
Merging (97, 99) into a new token 274
Merging (100, 262) into a new token 275


In [10]:
print("tokens length: ", len(tokens))
print("ids length: ", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length:  179
ids length:  113
compression ratio: 1.58X


El tokenizador es algo completamente independiente al LLM. Tiene su propio set de entrenamiento de texto (que PODRÍA ser diferente al del LLM) que utilizas para entrenarlo utilizando el Byte Pair Encoding (BPE) Algorithm. Después de esto es capaz de codificar y decodificar una secuencia de texto a tokens y viceversa.

Los datos para entrenar el tokenizador son importantes porque, por ejemplo, si tienes mucho texto en Japonés, significa que se formarán más tokens en Japonés.

## **Decoding**

No todos los bytes son decodificables, por ejejmplo el byte `128`. Entonces, si nuestro LLM no genera correctamente contenido puede ser que tengamos error en la decodificación. Para evitar esto, hacemos que, en vez de saltar un error, nos lo sustituya con `�`. Esto se consigue con `errors = "replace"`.

In [11]:
vocab = {idx : bytes([idx]) for idx in range(256)}

for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors = "replace")
    return text

decode([128])

'�'

In [12]:
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [13]:
merges

{(111, 32): 256,
 (101, 32): 257,
 (110, 32): 258,
 (97, 32): 259,
 (100, 257): 260,
 (44, 32): 261,
 (97, 114): 262,
 (260, 108): 263,
 (97, 110): 264,
 (111, 114): 265,
 (116, 105): 266,
 (117, 258): 267,
 (32, 263): 268,
 (99, 104): 269,
 (97, 261): 270,
 (110, 256): 271,
 (113, 117): 272,
 (101, 114): 273,
 (97, 99): 274,
 (100, 262): 275}

## **Encoding**
Para hacer los mergeos es importante hacerlos en el mismo orden en el que se han hecho antes, ya que unos mergeos utilizan a otros generados anteriormente, por ejemplo:

```
aaabdaaabac
```

Se sustituyen las `aa` por `Z`:
```
ZabdZabac
Z=aa
```

Se sustituyen las `ab` por `Y`:
```
ZYdZYac
Y=ab
Z=aa
```

Se sustituyen las `ZY` por `X`:
```
XdXac
X=ZY
Y=ab
Z=aa
```

Aquí no puedes utilizar el token `X` si no has generado antes el `Z` y el `Y`. Por esto hay que hacerlo en el mismo orden.

In [12]:
def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) > 2:
        stats = get_stats(tokens)
        pair = min(stats, key = lambda p : merges.get(p, float("inf"))) # Cogemos aquel que tiene el valor minimo de merges
        if pair not in merges:
            break # No se puede mergear nada
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
        
    return tokens

encode("hello world!")

[104, 101, 108, 108, 256, 119, 265, 108, 100, 33]

In [13]:
print(decode(encode("hello world!")))

hello world!


In [14]:
text = "hello world!"

text2 = decode(encode(text))
print(text == text2)

True


# **Complejizando el Tokenizador (GPT)**

En OpenAI se dieron cuenta de que si codificas únicamente de esta manera puede llegar a tener problemas ya que puedes tener palabras muy comunes como "perro" codificadas de manera diferente, por ejemplo: `perro.`, `perro!`, `perro?`...etc. Esto puede generar conflicto ya que tienes una misma palabra pero con diferentes codificaciones por la puntuación. Por esto mismo, OpenAI en [GPT2](https://github.com/openai/gpt-2/blob/master/src/encoder.py) implementó unos patrones regex para que esto no succediera. Primero lo que hace es splittear el texto según los patrones de texto y luego lo codifica para evitar los problemas mencionados anteriormente. Es decir, que solo puedes encontrar merges dentro de los elementos splitteados y cuando ya no puedas mergear más los elementos se juntarán en una lista.

In [15]:
import regex as re

gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""", re.IGNORECASE)

text = "Do you know where my 1st dog is?"
print(re.findall(gpt2pat, text))

['Do', ' you', ' know', ' where', ' my', ' 1', 'st', ' dog', ' is', '?']


Sin emabrgo, hay más cosas además de esta, OpenAI no ha revelado el código de entrenamiento del tokenizador por lo cuál no podemos estar seguros de qué más normas han puesto. Sin embargo, hay cosas como los espacios que aunque el splitter los dividide y varios de estos podrían componer un token no lo hacen, esto hablando de GPT2, en GPT4 si que están unidos. La librería de OpenAI de los tokenizadores es `tiktoken` pero no se pueden entrenar tokenizadores, es una librería solo de inferencia.

In [16]:
import tiktoken

## -- GPT2 (no une espacios)
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("    Hello World"))

## -- GPT4 (une espacios)
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("    Hello World"))

[220, 220, 220, 18435, 2159]
[262, 22691, 4435]


El patron de Regex también cambia para el [tokenizador de GPT4](https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py), se puede ver si bajas hasta `def cl100k_base()` y ves el pat_str. Pero no se sabe el por qué se hace así ya que no está documentado, tendrás sus razones pero no las podemos conocer.

## **Conociendo el tokenizador de GPT2**
Sabiendo todo esto, podemos entender el tokenizador ([`encoder.py`](https://github.com/openai/gpt-2/blob/master/src/encoder.py)) de GPT2, este depende de dos archivos (cómo se ve en el código): `encoder.json` y `vocab.bpe`. Estos archivos son para notros los objetos `merges` y `vocab` respectivamente. Lo vemos a continuación.

In [17]:
# !wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
# !wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

import json
import os

with open("tiktoken/encoder.json", "r") as f:
    encoder = json.load(f)

with open("tiktoken/vocab.bpe", "r", encoding = "utf-8") as f:
    bpe_data = f.read()

bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split("\n")[1:-1]]

Lo único confuso es que el tokenizador de GPT2 tiene un `byte_encoder` y un `byte_decoder`. Estos lo que hace es (Karpathy no lo entiende) es que primero hace una codificación a bytes y después un encode y después una decodificación a bytes antes de pasarlo a texto. El código es un poco diferente al nuestro pero algoritmicamente es igual a lo que se ha implementado anteriormente.

In [18]:
len(encoder) # 256 raw tokens + 50.000 merges + 1 token especial <|endoftext|>

50257

In [19]:
encoder["<|endoftext|>"] # Sirve para delimitar textos

50256

Este último token no está en [`encoder.py`](https://github.com/openai/gpt-2/blob/master/src/encoder.py) pero sí en [`tiktoken`](https://github.com/openai/tiktoken/tree/main?tab=MIT-1-ov-file). Se añade después del entrenamiento, es decir, es independiente al entrenamiento y se añade manualmente y los que queramos. No sirve solo para poder delimitar textos sino también conversaciones y mensajes (ver [tiktokenizer](https://tiktokenizer.vercel.app/)).
![Image](imgs/tiktokenizer.png)
Se pueden añadir tokens especiales a los modelos, sin embargo, es importante mencionar que será necesario hacerle un poco de cirugía al modelo, ya que vas a tener que añadirle uno (si solo añades un token especial) al vocab size y hacer que sea capaz de predecir un token más. Se suele coger un modelo preentrenado y se inicializan solo las nuevas conexiones de manera random para que aprenda durante el fine tuning.

# **Custom Tokenizer**
En este momento ya podemos entrenar nuestro propio tokenizador.

# **Sentencepiece**
Nos alejamos de `tiktoken`. Esta librería es de Google y se utiliza mucho porque te permite hacer entrenamientos e inferencias de tokenizadores. Se utiliza en Llama y Mistral (seguirá siendo así?). [Ver Código](https://github.com/google/sentencepiece)

La principal diferencia respecto a lo que hemos hecho es que cambian el orden de algunas cosas:
* Tiktoken: Codificamos a bytes de UTF-8 y después unimos bytes con BPE.
* Sentencepiece: No convierte a UTF-8 salvo que sea estrictamente necesario. Trabaja directamente con los code points de Unicode y recurre a UTF-8 solo en casos excepcionales (y es posible definir qué se considera "excepcional").

In [20]:
import sentencepiece as spm
import os

with open("examples/toy_text.txt", "r", encoding = "utf-8") as file:
    text = file.read()

A continuación se ve una aproximación a los ajustes utilizados para entrenar Llama 2. Notas importantes:
* Antes se solía normalizar mucho el texto, sin embargo, desde que aparecieron los LLM lo mejor es tocar lo menos posible.

In [21]:
options = dict(
    input = "examples/toy_text.txt",
    input_format = "text",
    model_prefix = "tok400", # El prefijo del filename
    model_type = "bpe",
    vocab_size = 400, # Obviamente llama tiene más
    normalization_rule_name = "identity",
    remove_extra_whitespaces = False,
    input_sentence_size = 200_000_000,
    max_sentence_length = 4192,
    seed_sentencepiece_size = 1_000_000,
    shuffle_input_sentence = True,
    character_coverage = 0.99995,
    byte_fallback = True, # Si esto es True, cuando no conozca algo pondrá el byte pero si es False pondra UNK 
    split_digits = True,
    split_by_unicode_script = True,
    split_by_whitespace = True,
    split_by_number = True,
    max_sentencepiece_length = 16,
    add_dummy_prefix = True, # Esto hace que trate "world" con el mismo token que "hello world" añadiendo un " " delante.
    allow_whitespace_only_pieces = True,
    unk_id = 0, # Debe existir
    bos_id = 1, # Beginning of Sentence
    eos_id = 2, # End of Sentence
    pad_id = -1, # Padding
    num_threads = 4
)

In [22]:
spm.SentencePieceTrainer.train(**options)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: examples/toy_text.txt
  input_format: text
  model_prefix: tok400
  model_type: BPE
  vocab_size: 400
  self_test_sample_size: 0
  character_coverage: 0.99995
  input_sentence_size: 200000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 4
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_priv

In [23]:
sp = spm.SentencePieceProcessor()
sp.load("tok400.model")
vocab = [[sp.id_to_piece(idx), idx ] for idx in range(sp.get_piece_size())]

In [24]:
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

Sentencepiece representa el vocab size así:
* Special tokens
* Byte tokes
* Merged tokens
* Individual codepoints: Si estos son muy raros se pueden ignorar (character_coverage)

In [25]:
ids = sp.encode("Sábado y domingo")
ids

[286, 198, 164, 384, 367, 301, 362, 381, 362, 301, 375, 273, 368]

In [26]:
print([sp.id_to_piece(idx) for idx in ids]) # Vemos que no detecta la tílde porque no está en su set de entrenamiento, recurre a los bytes

['▁S', '<0xC3>', '<0xA1>', 'b', 'a', 'do', '▁', 'y', '▁', 'do', 'm', 'ing', 'o']


# **Vocab Size**
El vocab size únicamente aparece en los embeddings y en la capa linear al final (para predecir el siguiente token). Aumentar el vocab size tiene dos pegas principales:
1. Aumentas el costo computacional
2. Cada token está menos representado (puede darse el caso de que el modelo no aprenda por falta de representatividad)
3. Más información está introducida en cada token y no se puede sacar toda la info que hay embedida

Hoy en día está en las decenas de miles o, en algunos casos, en los cientos de miles. Si queremos añadir nuevos tokens, tenemos que cambiarle el tamaño a la capa de los embeddings y a la capa linear del final y congelar el resto del modelo.

# **Multimodalidad**
Lo que haces no es cambiar el modelo, sino cambiar el tokenizador, es decir, "tokenizas" la imagen con una CNN y le pasas las representaciones como si fuera texto, ya que el modelo no entiende de texto o imágenes, sino de tokens.